<div align = center>
<h2> Feature selection</h2>
</div>

Consider the Titanic dataset for your practical work, and test feature-
engineering impact on the results.

- Get the data, and then give a short description using corresponding
statistical indicators.
- Clean the data by dealing with missing values, dropping unnecessary
features, standardizing the data...etc.
- Measure the information gain of each feature on the Survived
passengers.

- Test different configurations to build a classification model using K-
NN algorithm by: (a) testing different number of features, and (b) different number of K. (c) different similarity measure. <br>
    o Use precision for validating results. <br>
- Visualize and interpret results.
